# Import Required Libraries

In [ ]:
!pip install git+https://github.com/jacobgil/pytorch-grad-cam.git


!pip install lime

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from pytorch_grad_cam import GradCAM, GradCAMPlusPlus, EigenCAM, AblationCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from lime import lime_image
import zipfile
import os
from tqdm import tqdm

# Load and Prepare the Dataset

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root="/kaggle/input/bangladeshi-mango-leaf3/Image Dataset of Bangladeshi Mango Leaf/Root/Root/Original", transform=transform_train)
class_names = dataset.classes
num_classes = len(class_names)

print("Classes found:", class_names)
print("Number of classes:", num_classes)


train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

val_dataset.dataset.transform = transform_test
test_dataset.dataset.transform = transform_test

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Visualize Example Images for Each Class

In [ ]:
fig, axs = plt.subplots(1, num_classes, figsize=(15, 5))
displayed_classes = {class_name: False for class_name in class_names}

for images, labels in train_loader:
    for img, label in zip(images, labels):
        class_name = class_names[label]
        if not displayed_classes[class_name]:
            img = img.permute(1, 2, 0).numpy()
            img = (img * 0.5) + 0.5  # unnormalize
            axs[label].imshow(np.clip(img, 0, 1))
            axs[label].set_title(class_name)
            axs[label].axis('off')
            displayed_classes[class_name] = True

        if all(displayed_classes.values()):
            break
    if all(displayed_classes.values()):
        break

plt.show()

# Transfer Learning Models

In [ ]:
import torch.nn as nn
from torchvision import models

def get_transfer_model(model_name, num_classes):
    if model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

    elif model_name == "mobilenet_v2":
        model = models.mobilenet_v2(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

    elif model_name == "efficientnet_b0":
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

    elif model_name == "densenet121":
        model = models.densenet121(pretrained=True)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)

    elif model_name == "inception_v3":
        model = models.inception_v3(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)

    elif model_name == "resnet50":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)  

    else:
        raise ValueError(f"Model '{model_name}' not supported. Choose from vgg16, mobilenet_v2, efficientnet_b0, densenet121, inception_v3, resnet50.")

    return model

# Training and Early Stopping

In [ ]:
class EarlyStopping:

    def __init__(self, patience=5):

        self.patience = patience
        self.counter = 0
        self.best_loss = np.inf

    def check_early_stop(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

# Transfer Learning Example using DenseNet121

In [ ]:
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

num_epochs = 50

model = get_transfer_model('densenet121', num_classes).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

early_stopping = EarlyStopping(patience=5)

train_losses, val_losses = [], []

scaler = GradScaler()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    model.train()
    train_loss = 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation", leave=False):
            images, labels = images.to('cuda'), labels.to('cuda')
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f"Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

    if early_stopping.check_early_stop(avg_val_loss):
        print("Early stopping triggered.")
        break

# Save model
torch.save(model.state_dict(), 'transfer_learning_densenet121.pth')
print("Model saved as 'transfer_learning_densenet121.pth'")

In [ ]:
# Plotting loss curves

plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title("Training and Validation Loss for DenseNet-121")
plt.show()

# Model Evaluation and Metrics Calculation

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():  
    for images, labels in test_loader:
        
        images, labels = images.to('cuda'), labels.to('cuda')

        outputs = model(images)

        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())  
        all_labels.extend(labels.cpu().numpy())    

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# XAI - Grad-CAM, Grad-CAM++, Eigen-CAM, Ablation-CAM

In [ ]:
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

def get_transfer_model(model_name, num_classes):
    if model_name == "densenet121":
        model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    else:
        raise ValueError(f"Model '{model_name}' not supported.")
    return model

model = get_transfer_model('densenet121', num_classes).to('cuda')
model.load_state_dict(torch.load('transfer_learning_densenet121.pth'))  
model.eval()

sample_idx = 0  
sample_image, true_label = test_dataset[sample_idx]
sample_image = sample_image.unsqueeze(0).to('cuda')  

original_image_np = sample_image.squeeze(0).permute(1, 2, 0).cpu().numpy()
original_image_np = (original_image_np * 0.5) + 0.5  
original_image_np = np.clip(original_image_np, 0, 1)

target_layers = [model.features.norm5]  

gradcam = GradCAM(model=model, target_layers=target_layers)
gradcam_plus_plus = GradCAMPlusPlus(model=model, target_layers=target_layers)
eigen_cam = EigenCAM(model=model, target_layers=target_layers)
ablation_cam = AblationCAM(model=model, target_layers=target_layers)

with torch.no_grad():
    outputs = model(sample_image)
    predicted_class = outputs.argmax().item()
    predicted_class_name = class_names[predicted_class]
    true_class_name = class_names[true_label]

target = [ClassifierOutputTarget(predicted_class)]

gradcam_heatmap = gradcam(input_tensor=sample_image, targets=target)[0]
gradcam_pp_heatmap = gradcam_plus_plus(input_tensor=sample_image, targets=target)[0]
eigen_cam_heatmap = eigen_cam(input_tensor=sample_image, targets=target)[0]
ablation_cam_heatmap = ablation_cam(input_tensor=sample_image, targets=target)[0]

gradcam_result = show_cam_on_image(original_image_np, gradcam_heatmap, use_rgb=True)
gradcam_pp_result = show_cam_on_image(original_image_np, gradcam_pp_heatmap, use_rgb=True)
eigen_cam_result = show_cam_on_image(original_image_np, eigen_cam_heatmap, use_rgb=True)
ablation_cam_result = show_cam_on_image(original_image_np, ablation_cam_heatmap, use_rgb=True)

In [ ]:
# Plot the results
plt.figure(figsize=(20, 5))
plt.subplot(1, 5, 1)
plt.imshow(original_image_np)
plt.title(f"Original Image\n(True: {true_class_name}, Pred: {predicted_class_name})", fontsize=10)
plt.axis("off")

plt.subplot(1, 5, 2)
plt.imshow(gradcam_result)
plt.title(f"Grad-CAM\n(Predicted: {predicted_class_name})", fontsize=10)
plt.axis("off")

plt.subplot(1, 5, 3)
plt.imshow(gradcam_pp_result)
plt.title(f"Grad-CAM++\n(Predicted: {predicted_class_name})", fontsize=10)
plt.axis("off")

plt.subplot(1, 5, 4)
plt.imshow(eigen_cam_result)
plt.title(f"Eigen-CAM\n(Predicted: {predicted_class_name})", fontsize=10)
plt.axis("off")

plt.subplot(1, 5, 5)
plt.imshow(ablation_cam_result)
plt.title(f"Ablation-CAM\n(Predicted: {predicted_class_name})", fontsize=10)
plt.axis("off")

plt.tight_layout()
plt.show()

# LIME

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
from PIL import Image

def batch_predict(images):
    model.eval()
    
    batch = torch.stack([
        transform_test(Image.fromarray((image * 255).astype(np.uint8))) 
        for image in images
    ], dim=0).to('cuda')
    
    with torch.no_grad():
        logits = model(batch)
    
    return torch.nn.functional.softmax(logits, dim=1).cpu().numpy()

explainer = lime_image.LimeImageExplainer()

lime_explanation = explainer.explain_instance(
    original_image_np,
    batch_predict,
    top_labels=1,       
    hide_color=0,
    num_samples=100    
)

lime_image, lime_mask = lime_explanation.get_image_and_mask(
    label=predicted_class,
    positive_only=True,
    hide_rest=False,
    num_features=10,
    min_weight=0.01
)
lime_image = mark_boundaries(lime_image, lime_mask)

# Display the original and LIME result
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(original_image_np)
plt.title(f"Original Image\n(True: {true_class_name}, Pred: {predicted_class_name})")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(lime_image)
plt.title(f"LIME Explanation\n(Predicted: {predicted_class_name})")
plt.axis("off")

plt.tight_layout()
plt.show()